# Lab 5


Matrix Representation: In this lab you will be creating a simple linear algebra system. In memory, we will represent matrices as nested python lists as we have done in lecture. In the exercises below, you are required to explicitly test every feature you implement, demonstrating it works.

1. Create a `matrix` class with the following properties:
    * It can be initialized in 2 ways:
        1. with arguments `n` and `m`, the size of the matrix. A newly instanciated matrix will contain all zeros.
        2. with a list of lists of values. Note that since we are using lists of lists to implement matrices, it is possible that not all rows have the same number of columns. Test explicitly that the matrix is properly specified.
    * Matrix instances `M` can be indexed with `M[i][j]` and `M[i,j]`.
    * Matrix assignment works in 2 ways:
        1. If `M_1` and `M_2` are `matrix` instances `M_1=M_2` sets the values of `M_1` to those of `M_2`, if they are the same size. Error otherwise.
        2. In example above `M_2` can be a list of lists of correct size.


2. Add the following methods:
    * `shape()`: returns a tuple `(n,m)` of the shape of the matrix.
    * `transpose()`: returns a new matrix instance which is the transpose of the matrix.
    * `row(n)` and `column(n)`: that return the nth row or column of the matrix M as a new appropriately shaped matrix object.
    * `to_list()`: which returns the matrix as a list of lists.
    *  `block(n_0,n_1,m_0,m_1)` that returns a smaller matrix located at the n_0 to n_1 columns and m_0 to m_1 rows. 
    * (Extra credit) Modify `__getitem__` implemented above to support slicing.
        

3. Write functions that create special matrices (note these are standalone functions, not member functions of your `matrix` class):
    * `constant(n,m,c)`: returns a `n` by `m` matrix filled with floats of value `c`.
    * `zeros(n,m)` and `ones(n,m)`: return `n` by `m` matrices filled with floats of value `0` and `1`, respectively.
    * `eye(n)`: returns the n by n identity matrix.

4. Add the following member functions to your class. Make sure to appropriately test the dimensions of the matrices to make sure the operations are correct.
    * `M.scalarmul(c)`: a matrix that is scalar product $cM$, where every element of $M$ is multiplied by $c$.
    * `M.add(N)`: adds two matrices $M$ and $N$. Don’t forget to test that the sizes of the matrices are compatible for this and all other operations.
    * `M.sub(N)`: subtracts two matrices $M$ and $N$.
    * `M.mat_mult(N)`: returns a matrix that is the matrix product of two matrices $M$ and $N$.
    * `M.element_mult(N)`: returns a matrix that is the element-wise product of two matrices $M$ and $N$.
    * `M.equals(N)`: returns true/false if $M==N$.

5. Overload python operators to appropriately use your functions in 4 and allow expressions like:
    * 2*M
    * M*2
    * M+N
    * M-N
    * M*N
    * M==N
    * M=N


6. Demonstrate the basic properties of matrices with your matrix class by creating two 2 by 2 example matrices using your Matrix class and illustrating the following:

$$
(AB)C=A(BC)
$$
$$
A(B+C)=AB+AC
$$
$$
AB\neq BA
$$
$$
AI=A
$$

In [7]:
class Matrix:
    def __init__(self, n=None, m=None):
        if isinstance(n, list):
            self.data = n
            self.n = len(n)
            self.m = len(n[0]) if n else 0
            if not all(len(row) == self.m for row in n):
                print("Error All rows must have the same number of columns.")
                self.data = None
                self.n = self.m = 0
        elif isinstance(n, int) and isinstance(m, int):
            self.n = n
            self.m = m
            self.data = [[0 for _ in range(m)] for _ in range(n)]
        else:
            print("Error Invalid initialization parameters.")
            self.data = None
            self.n = self.m = 0
    
    def __getitem__(self, index):
        if self.data is None:
            print("Error Matrix is not properly initialized.")
            return None
        if isinstance(index, tuple):
            return self.data[index[0]][index[1]]
        elif isinstance(index, int):
            return self.data[index]
        else:
            print("Error Index must be an int or a tuple.")
            return None

    def __setitem__(self, index, value):
        if self.data is None:
            print("Error Matrix is not properly initialized.")
            return
        if isinstance(index, tuple):
            self.data[index[0]][index[1]] = value
        else:
            self.data[index] = value

    def shape(self):
        return (self.n, self.m)

    def transpose(self):
        return Matrix([[self.data[j][i] for j in range(self.n)] for i in range(self.m)])

    def row(self, n):
        return Matrix([self.data[n]]) if n < self.n else None

    def column(self, n):
        return Matrix([[self.data[i][n]] for i in range(self.n)]) if n < self.m else None

    def to_list(self):
        return self.data

    def block(self, n_0, n_1, m_0, m_1):
        if n_0 < self.n and m_0 < self.m:
            return Matrix([row[m_0:m_1] for row in self.data[n_0:n_1]])
        print("Error Block indices out of bounds.")
        return None

    def scalarmul(self, c):
        return Matrix([[self.data[i][j] * c for j in range(self.m)] for i in range(self.n)])

    def add(self, N):
        if self.shape() != N.shape():
            print("Error Matrices must have the same dimensions for addition.")
            return None
        return Matrix([[self.data[i][j] + N[i][j] for j in range(self.m)] for i in range(self.n)])

    def sub(self, N):
        if self.shape() != N.shape():
            print("Error Matrices must have the same dimensions for subtraction.")
            return None
        return Matrix([[self.data[i][j] - N[i][j] for j in range(self.m)] for i in range(self.n)])

    def mat_mult(self, N):
        if self.m != N.n:
            print("Error Matrices are not aligned for multiplication.")
            return None
        return Matrix([[sum(self.data[i][k] * N[k][j] for k in range(self.m)) for j in range(N.m)] for i in range(self.n)])

    def element_mult(self, N):
        if self.shape() != N.shape():
            print("Error Matrices must have the same dimensions for element-wise multiplication.")
            return None
        return Matrix([[self.data[i][j] * N[i][j] for j in range(self.m)] for i in range(self.n)])

    def equals(self, N):
        return self.shape() == N.shape() and all(self.data[i][j] == N[i][j] for i in range(self.n) for j in range(self.m))

    def __eq__(self, N):
        return self.equals(N)

    def __add__(self, N):
        return self.add(N)

    def __sub__(self, N):
        return self.sub(N)

    def __mul__(self, N):
        if isinstance(N, (int, float)):
            return self.scalarmul(N)
        elif isinstance(N, Matrix):
            return self.mat_mult(N)
        else:
            print("Error Unsupported operand type(s) for *")
            return None

    def __rmul__(self, N):
        return self.scalarmul(N)

    def __repr__(self):
        return f"Matrix({self.data})"

def constant(n, m, c):
    return Matrix([[c for _ in range(m)] for _ in range(n)])

def zeros(n, m):
    return Matrix(n, m)

def ones(n, m):
    return Matrix([[1 for _ in range(m)] for _ in range(n)])

def eye(n):
    return Matrix([[1 if i == j else 0 for j in range(n)] for i in range(n)])


In [6]:
if __name__ == "__main__":
    A = Matrix([[1, 2], [3, 4]])
    B = Matrix([[5, 6], [7, 8]])
    C = Matrix([[2, 2], [2, 2]])
    
    print("A + B:", A + B)
    print("A - B:", A - B)
    print("A * B:", A * B)
    print("2 * A:", 2 * A)
    print("A * 2:", A * 2)
    print("A equals B:", A.equals(B))
    print("Transpose of A:", A.transpose())
    print("Shape of A:", A.shape())
    print("Row 1 of A:", A.row(1))
    print("Column 0 of A:", A.column(0))
    print("Block of A from (0, 2) to (0, 2):", A.block(0, 2, 0, 2))
    
    print("Demonstrating Associativity: (AB)C = A(BC)")
    print("A * B * C:", A * (B * C))
    print("A * (B * C):", (A * B) * C)

    print("Demonstrating Distributivity: A(B + C) = AB + AC")
    print("A * (B + C):", A * (B + C))
    print("A * B + A * C:", (A * B) + (A * C))

    print("Demonstrating Non-Commutativity: AB != BA")
    print("A * B:", A * B)
    print("B * A:", B * A)

    print("Demonstrating Identity: AI = A")
    I = eye(2)
    print("A * I:", A * I)


A + B: Matrix([[6, 8], [10, 12]])
A - B: Matrix([[-4, -4], [-4, -4]])
A * B: Matrix([[19, 22], [43, 50]])
2 * A: Matrix([[2, 4], [6, 8]])
A * 2: Matrix([[2, 4], [6, 8]])
A equals B: False
Transpose of A: Matrix([[1, 3], [2, 4]])
Shape of A: (2, 2)
Row 1 of A: Matrix([[3, 4]])
Column 0 of A: Matrix([[1], [3]])
Block of A from (0, 2) to (0, 2): Matrix([[1, 2], [3, 4]])
Demonstrating Associativity: (AB)C = A(BC)
A * B * C: Matrix([[82, 82], [186, 186]])
A * (B * C): Matrix([[82, 82], [186, 186]])
Demonstrating Distributivity: A(B + C) = AB + AC
A * (B + C): Matrix([[25, 28], [57, 64]])
A * B + A * C: Matrix([[25, 28], [57, 64]])
Demonstrating Non-Commutativity: AB != BA
A * B: Matrix([[19, 22], [43, 50]])
B * A: Matrix([[23, 34], [31, 46]])
Demonstrating Identity: AI = A
A * I: Matrix([[1, 2], [3, 4]])
